In [1]:
!pip uninstall optimum-neuron -y

Found existing installation: optimum-neuron 0.1.1.dev0
Uninstalling optimum-neuron-0.1.1.dev0:
  Successfully uninstalled optimum-neuron-0.1.1.dev0


In [2]:
!pip install git+https://github.com/huggingface/optimum-neuron.git

  Cloning https://github.com/huggingface/optimum-neuron.git to /tmp/pip-req-build-wvr6oyol
  Running command git clone -q https://github.com/huggingface/optimum-neuron.git /tmp/pip-req-build-wvr6oyol
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 7.0 MB 28.9 MB/s eta 0:00:01
  Created wheel for optimum-neuron: filename=optimum_neuron-0.1.1.dev0-py3-none-any.whl size=45733 sha256=fee5e1e3e27ee49917af0b5803448fdbce19b82849d640357f4261889d1f85dd
  Stored in directory: /tmp/pip-ephem-wheel-cache-oau6ozjb/wheels/a9/44/8d/8dbc0bb2e5ff2368394043699944ee69f70485e4852f4bfdd2
Successfully built optimum-neuron


In [3]:
!neuron-ls

instance-type: trn1.2xlarge
instance-id: i-0570615e41700a481
+--------+--------+--------+---------+
| NEURON | NEURON | NEURON |   PCI   |
| DEVICE | CORES  | MEMORY |   BDF   |
+--------+--------+--------+---------+
| 0      | 2      | 32 GB  | 00:1e.0 |
+--------+--------+--------+---------+


## Preprocess dataset

In [2]:
from datasets import load_dataset

# Dataset id from huggingface.co/dataset
dataset_id = "banking77"
# Model id to load the tokenizer
model_id = "bert-base-uncased"
save_dataset_path = "dataset"


# Load raw dataset
raw_dataset = load_dataset(dataset_id)

print(f"Train dataset size: {len(raw_dataset['train'])}")
print(f"Test dataset size: {len(raw_dataset['test'])}")

# Train dataset size: 10003
# Test dataset size: 3080
from random import randrange

random_id = randrange(len(raw_dataset['train']))
print(raw_dataset['train'][random_id])
# {'text': 'How can I change my PIN without going to the bank?', 'label': 21}

import os
from transformers import AutoTokenizer


# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Tokenize helper function
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True,return_tensors="pt")

# Tokenize dataset
raw_dataset =  raw_dataset.rename_column("label", "labels") # to match Trainer
tokenized_dataset = raw_dataset.map(tokenize, batched=True, remove_columns=["text"])
tokenized_dataset = tokenized_dataset.with_format("torch")

print(tokenized_dataset["train"].features.keys())
# dict_keys(['input_ids', 'token_type_ids', 'attention_mask','lable'])
# save dataset to disk
tokenized_dataset["train"].save_to_disk(os.path.join(save_dataset_path,"train"))
tokenized_dataset["test"].save_to_disk(os.path.join(save_dataset_path,"eval"))

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset banking77 (/home/ubuntu/.cache/huggingface/datasets/banking77/default/1.1.0/ff44c4421d7e70aa810b0fa79d36908a38b87aff8125d002cd44f7fcd31f493c)
100%|██████████| 2/2 [00:00<00:00, 130.36it/s]
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/banking77/default/1.1.0/ff44c4421d7e70aa810b0fa79d36908a38b87aff8125d002cd44f7fcd31f493c/cache-084cb9babe899b20.arrow
Loading cached processed dataset at /home/ubuntu/.cache/huggingface/datasets/banking77/default/1.1.0/ff44c4421d7e70aa810b0fa79d36908a38b87aff8125d002cd44f7fcd31f493c/cache-5f7f794fe0ef7b57.arrow


Train dataset size: 10003
Test dataset size: 3080
{'text': 'Can I change my address?', 'label': 30}
dict_keys(['labels', 'input_ids', 'token_type_ids', 'attention_mask'])


## precompiple 

In [3]:
!neuron_parallel_compile python3 scripts/train.py --model_id bert-base-uncased --per_device_train_batch_size 8 

2023-03-20 12:50:18.000261: INFO ||PARALLEL_COMPILE||: Running trial run (add option to terminate trial run early; also ignore trial run's generated outputs, i.e. loss, checkpoints)
is precompilation: 1
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly

`2023-03-13 08:24:52.000851: ERROR ||PARALLEL_COMPILE||: parallel compilation with neuronx-cc exited with error.Received error code: 137`

=> OOM? but training with BS 8 works. 

```
2023-03-13 08:26:28.000757: INFO ||PARALLEL_COMPILE||: Total graphs: 2
2023-03-13 08:26:28.000757: INFO ||PARALLEL_COMPILE||: Total successful compilations: 0
2023-03-13 08:26:28.000757: INFO ||PARALLEL_COMPILE||: Total failed compilations: 2
```

monitoring `htop` has shown that we ran out of CPU Memory -> (30.8GB)

`free -h`

shows no swap 


```bash
# add swap 10GB of swap
sudo fallocate -l 20G /swapfile
# change permissions
sudo chmod 600 /swapfile
# mark the file as swap space
sudo mkswap /swapfile
# enable swap 
sudo swapon /swapfile
# make swap file permanent
# echo '/swapfile none swap sw 0 0' | sudo tee -a /etc/fstab
```

20 GB was not enough! 🤯


![img](./im.png)

switch swap: https://askubuntu.com/questions/1264568/increase-swap-in-20-04

# train

In [7]:
!rm -r /var/tmp/neuron-compile-cache

In [6]:
pip install huggingface_hub --upgrade

     |████████████████████████████████| 199 kB 39.9 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [5]:
import huggingface_hub

huggingface_hub.__version__

'0.12.1'

In [7]:
!CUSTOM_CACHE_REPO=philschmid/neuron-cache python3 scripts/train.py  --model_id bert-base-cased --per_device_train_batch_size 8 

is precompilation: None
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from

# Distrubted test 

In [1]:
!torchrun --nproc_per_node=2 scripts/train.py --model_id bert-base-cased --per_device_train_batch_size 8 --bf16 True

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
is precompilation: None
is precompilation: None
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).


In [3]:
!CUSTOM_CACHE_REPO=philschmid/neuron-cache torchrun --nproc_per_node=2 scripts/train.py --model_id bert-base-cased --per_device_train_batch_size 8 --bf16 True

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
is precompilation: None
is precompilation: None
Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).


1760.8812s -> 591.1733s 